# Example of building and training a simple Neural Network
In this example, a simple neural network is created using the `neural` framework.
The purpose of this network is to emulate the following function:
$$y = \sin\left(\frac{\pi}{2}x\right), x\in [-1, 1]$$

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import time

from neural import Tensor, nn, optim

## Defining the training function

In [ ]:
def training(x):
    return np.sin(np.pi/2*x)

## Defining the Neural Network architecture

In [ ]:
class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 1)
        
    def forward(self, x):
        x = nn.Tanh()(self.fc1(x))
        x = nn.Tanh()(self.fc2(x))
        return x
    
model = Network()

## Performance before training

In [ ]:
def evaluatePerformance(input_, title=""):
    correctOutput = training(input_)
    networkOutput = model(Tensor(input_))

    plt.figure(figsize=(12, 8))
    plt.plot(input_, networkOutput, label="Network output")
    plt.plot(input_, correctOutput, label="Correct output")
    plt.title(title)
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
input_ = np.linspace(-1, 1, 100).reshape([-1, 1])

In [ ]:
evaluatePerformance(input_, title="Before training")

## Choosing training criterion (loss function) and optimizer

In [ ]:
# Loss function
reduction = "mean"
criterion = nn.L1Loss(reduction=reduction)

# Optimizer setup
lr = 0.03
momentum = 0.9

optimizer = optim.SGD(
    model.parameters(),
    lr=lr,
    momentum=momentum)

## Training

In [ ]:
# Training samples
trainIn = 2*np.random.rand(10000) - 1
# Training target
target = training(trainIn)

In [ ]:
batchSize = 10

numBatches = trainIn.size // batchSize
numTraining = int(numBatches * batchSize)

print(f"Batch size: {batchSize}")
print(f"Total number of train samples: {numTraining}")
print(f"Total number of batches: {numBatches}")

# Reshaping training data
trainIn_ = Tensor(trainIn[:numTraining].reshape(numBatches, -1, 1))

In [ ]:
lossTrack = np.zeros(numBatches)

for i, x in enumerate(trainIn_):
    optimizer.zeroGrad()
    out = model(x)
    loss = criterion(out, training(x))
    loss.backward()
    lossTrack[i] = loss.item()
    optimizer.step()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(lossTrack)
plt.ylabel("Loss")
plt.xlabel("Batches processed")
plt.title("Training loss")
plt.grid()
plt.show()

### Saving the trained model

In [ ]:
nn.Module.save(model, "simple.pkl")

Saved module can be loaded with
```python
model = nn.Module.load("simple.pkl")
```

## Performance evaluation

In [ ]:
evaluatePerformance(input_, title="After training")

In [ ]:
networkOutput = model(Tensor(input_))
correctOutput = training(input_)

error = networkOutput - correctOutput
print(f"Maximum absolute error: {np.max(np.abs(error)):.2f}")